In [1]:
import os
from dotenv import load_dotenv

dotenv_path = '.env'
load_dotenv(dotenv_path)

# Print out the environment variables loaded
for key, value in os.environ.items():
    if 'OPENAI' in key:
        print("OpenAI Key founded")
        #print(f"{key}: {value}")

openai_api_key = os.getenv('OPENAI_API_KEY')

#print(f"OpenAI API Key: {openai_api_key}")

OpenAI Key founded


## Simple Agent

In [55]:
#!pip install numexpr
#!pip install -U langchain-openai
!pip install langchain --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.67
    Uninstalling langsmith-0.1.67:
      Successfully uninstalled langsmith-0.1.67
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.3
    Uninstalling langchain-core-0.2.3:
      Successfully uninstalled langchain-core-0.2.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.1
    Uninstalling langchain-0.2.1:
      Successfully uninstalled langchain-0.2.1


In [56]:
from langchain_openai import OpenAI
from langchain.agents import load_tools, AgentType, create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

llm = OpenAI()

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
print(tools)

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<openai.resources.completions.Completions o

In [25]:
# Define the prompt template
template = '''
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
'''

# Create the prompt from the template
prompt = PromptTemplate.from_template(template)

# Initialize the agent using create_react_agent function
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

print(agent)

# Wrap the agent in an AgentExecutor to control max_iterations
agent_executor = AgentExecutor(agent=agent, tools=tools, max_iterations=10)
print(agent_executor)

first=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
}) middle=[PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Calculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.', 'tool_names': 'Calculator'}, template='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observa

In [26]:
# Example input to run the agent
input_question = "What is the square root of 16?"

# Run the agent
response = agent_executor.invoke({
    "input": input_question,
    "messages": [HumanMessage(content=input_question)]
})

print(response)

{'input': 'What is the square root of 16?', 'messages': [HumanMessage(content='What is the square root of 16?')], 'output': '4.0'}
